In [1]:
import numpy
import keras
from keras_rcnn import datasets, models, preprocessing, utils
from keras_rcnn.datasets import shape
import json
import os
import random
import numpy as np
import tensorflow as tf
!dir

Using TensorFlow backend.


 Volume in drive C is OS
 Volume Serial Number is 2AEF-42D6

 Directory of C:\dev\datascienceworkshop-pneumonia

13/10/2018  15:30    <DIR>          .
13/10/2018  15:30    <DIR>          ..
18/09/2018  21:40             1,328 .gitignore
13/10/2018  13:52    <DIR>          .ipynb_checkpoints
14/09/2018  00:18    <DIR>          .vscode
13/10/2018  13:34            25,589 CheckEnvironment.ipynb
27/09/2018  22:05               491 config.py
13/10/2018  13:34         3,603,514 create_patient_JSONs.ipynb
13/10/2018  13:34    <DIR>          data
27/09/2018  22:05    <DIR>          data_preprocessing
09/09/2018  20:42           387,595 kernel.ipynb
13/10/2018  13:34           228,675 kernel_LG.ipynb
13/10/2018  15:30    <DIR>          logs
14/09/2018  00:18    <DIR>          models
13/10/2018  15:30            24,600 obj_detection_rcnn.ipynb
13/10/2018  15:29    <DIR>          output
13/10/2018  13:34            65,775 Pneumonia_Training.ipynb
09/09/2018  20:38                31 README.md
09/0

In [2]:
DATA_PATH = r'data'
JSON_FILE = os.path.join(DATA_PATH,r'train_positive.json')

In [3]:
#load data

with open(JSON_FILE) as j:
    data = json.load(j)
    
data[0]

{'image': {'checksum': '14a05c5cd4498a9beb3bc9ca555c8cd0',
  'pathname': 'data/train_images/positive/4525bf0b-20b4-4f4f-9149-e32632812d89.png',
  'shape': {'r': 1024, 'c': 1024, 'channels': 1}},
 'objects': [{'bounding_box': {'minimum': {'r': 301, 'c': 403},
    'maximum': {'r': 491, 'c': 757}},
   'category': 'sick'},
  {'bounding_box': {'minimum': {'r': 533, 'c': 359},
    'maximum': {'r': 788, 'c': 755}},
   'category': 'sick'}]}

In [4]:
data_shuffled = data[:]

portion = int(0.8*len(data_shuffled))

#shuffle  list
random.seed(55)
random.shuffle(data_shuffled)

training_dictionary = data_shuffled[:20]
test_dictionary = data_shuffled[20:30]

epoch_size = len(training_dictionary)
epoch_size

20

In [5]:
categories = {"sick": 1}

generator = preprocessing.ObjectDetectionGenerator()

generator = generator.flow_from_dictionary(
    dictionary=training_dictionary,
    categories=categories,
    target_size=(224, 224),
    color_mode='rgb'
)

validation_data = preprocessing.ObjectDetectionGenerator()

validation_data = validation_data.flow_from_dictionary(
    dictionary=test_dictionary,
    categories=categories,
    target_size=(224, 224),
    color_mode='rgb'
)

Create RCNN model

[https://github.com/broadinstitute/keras-rcnn](https://github.com/broadinstitute/keras-rcnn)

In [6]:
# keras_rcnn.
model = models.RCNN((224, 224, 3), ["pneumonia"])

optimizer = keras.optimizers.Adam()

model.compile(optimizer)

## Keras callbacks  
[https://keras.io/callbacks/](https://keras.io/callbacks/)

Import callbacks

In [7]:
from keras.callbacks import Callback, TerminateOnNaN, TensorBoard, LearningRateScheduler, ModelCheckpoint
import keras.backend as K

In [8]:
class SGDRScheduler(TensorBoard):
    '''Cosine annealing learning rate scheduler with periodic restarts.
    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}
        
        # tensorboard
        log_dir = './logs'
        super(SGDRScheduler, self).__init__(log_dir, histogram_freq=0, batch_size=16, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
        self.lr_log_dir = log_dir

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()
        
        # tensorboard
        lr = self.clr()
        summary = tf.Summary(value=[tf.Summary.Value(tag='lr', simple_value=lr),
                                    tf.Summary.Value(tag='loss', simple_value=logs.get('loss')),
                                    tf.Summary.Value(tag='val_loss', simple_value=logs.get('val_loss'))])
        self.lr_writer.add_summary(summary, epoch)
        self.lr_writer.flush()
        super(SGDRScheduler, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)
        
        # tensorboard
        super(SGDRScheduler, self).on_train_end(logs)
        self.lr_writer.close()
    
    def set_model(self, model):
        self.lr_writer = tf.summary.FileWriter(self.lr_log_dir)
        super(SGDRScheduler, self).set_model(model)

    

Instantiate callbacks

In [9]:
# Loss history
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history = LossHistory()

# terminate on NaN
term_nan = TerminateOnNaN()    

# checkpoint
checkpoint_filename = os.path.join('C:\dev\datascienceworkshop-pneumonia\output', 'weights.{epoch:02d}-{val_loss:.2f}.hdf5')
checkpoint = ModelCheckpoint(checkpoint_filename, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=True, mode='auto', period=1)

# Tensorboard
tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

# Learning rate reset
def UpdateLR(i, lr):
    if (i == 0):
        return 0.001
    elif(i < 10):
        return lr*0.9
    elif(i < 20):
        return lr*0.8
lrs = LearningRateScheduler(schedule=UpdateLR, verbose=1)

# Cosine annealing reset
batch_size=16
sgdr = SGDRScheduler(min_lr=1e-5,
                         max_lr=1e-3,
                         steps_per_epoch=np.ceil(epoch_size/batch_size),
                         lr_decay=0.9,
                         cycle_length=5,
                         mult_factor=1.5)

## Run model

On a terminal, run:  

```tensorboard --logdir=C:/dev/datascienceworkshop-pneumonia/logs```

In [3]:
import webbrowser
webbrowser.open('http://desktop-in5570r:6006')

True

In [10]:
model.fit_generator(
    epochs=10,
    generator=generator,
    validation_data=validation_data,
    callbacks=[sgdr, checkpoint]
)

C:\Users\loren\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10


C:\Users\loren\Anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
C:\Users\loren\Anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
C:\Users\loren\Anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


20/20 [==============================] - 27s 1s/step - loss: 65.4348 - val_loss: 2241778.5192
Epoch 2/10
20/20 [==============================] - 20s 976ms/step - loss: 2.6959 - val_loss: 1915414.1318
Epoch 3/10
20/20 [==============================] - 19s 973ms/step - loss: 1.8244 - val_loss: 1363598.2264
Epoch 4/10
20/20 [==============================] - 19s 970ms/step - loss: 1.5735 - val_loss: 600140.3603
Epoch 5/10
20/20 [==============================] - 20s 981ms/step - loss: 1.3621 - val_loss: 33680.4724
Epoch 6/10
20/20 [==============================] - 19s 974ms/step - loss: 1.3056 - val_loss: 5704.9190
Epoch 7/10
20/20 [==============================] - 19s 970ms/step - loss: 1.3472 - val_loss: 351256.8469
Epoch 8/10
20/20 [==============================] - 20s 1s/step - loss: 1.2238 - val_loss: 2569.0207
Epoch 9/10
20/20 [==============================] - 20s 1s/step - loss: 1.2862 - val_loss: 9735.4608
Epoch 10/10
20/20 [==============================] - 20s 1s/step - lo